# **Prueba 1:**

In [38]:
from datetime import datetime

class Producto:
    def __init__(self, nombre, fecha_vencimiento=None, perecible=False):
        self.nombre = nombre
        # Convertir la fecha de vencimiento a un objeto datetime si se proporciona
        self.fecha_vencimiento = datetime.strptime(fecha_vencimiento, "%Y-%m-%d") if fecha_vencimiento else None
        self.perecible = perecible

class Nodo:
    def __init__(self, tipo):
        self.tipo = tipo  # Tipo de producto (cárnicos, lácteos, conservas, etc.)
        self.productos = []  # Lista de productos de este tipo
        self.hijos = []  # Subcategorías o más productos

    def agregar_producto(self, producto):
        self.productos.append(producto)
        if producto.perecible:
            # Ordenar productos perecibles por fecha de vencimiento, más próximo primero
            self.productos.sort(key=lambda x: x.fecha_vencimiento)

    def agregar_hijo(self, nodo):
        self.hijos.append(nodo)

    def buscar_producto_por_tipo(self, tipo):
        # Búsqueda por tipo de producto en el árbol
        if self.tipo == tipo:
            return self.productos
        for hijo in self.hijos:
            resultado = hijo.buscar_producto_por_tipo(tipo)
            if resultado:
                return resultado
        return []

    def buscar_productos_perecibles(self):
        """Devuelve una lista de productos perecibles en este nodo y sus hijos."""
        productos_perecibles = []
        
        # Agregar productos perecibles del nodo actual
        for producto in self.productos:
            if producto.perecible:
                productos_perecibles.append(producto)
        
        # Agregar productos perecibles de los hijos
        for hijo in self.hijos:
            productos_perecibles.extend(hijo.buscar_productos_perecibles())
            
        return productos_perecibles
    
    def buscar_producto_por_vencimiento(self, fecha):
        # Convertir la fecha de búsqueda a un objeto datetime
        fecha_busqueda = datetime.strptime(fecha, "%Y-%m-%d")
        productos_vencidos = []
        for producto in self.productos:
            if producto.perecible and producto.fecha_vencimiento <= fecha_busqueda:
                productos_vencidos.append(producto)
        for hijo in self.hijos:
            productos_vencidos.extend(hijo.buscar_producto_por_vencimiento(fecha))
        return productos_vencidos


# Crear un árbol genérico de productos
arbol_productos = Nodo("Almacén")

# Crear nodos para tipos de productos
frutas = Nodo("Frutas")
verduras = Nodo("Verduras")
conservas = Nodo("Conservas")
granos = Nodo("Granos")

# Agregar los nodos de tipos de productos al árbol principal
arbol_productos.agregar_hijo(frutas)
arbol_productos.agregar_hijo(verduras)
arbol_productos.agregar_hijo(conservas)
arbol_productos.agregar_hijo(granos)
 
# Agregar productos a cada tipo
frutas.agregar_producto(Producto("Manzana", "2024-11-10", perecible=True))
frutas.agregar_producto(Producto("Melocoton", "2024-11-15", perecible=True))
frutas.agregar_producto(Producto("Platano", "2024-12-01", perecible=True))
verduras.agregar_producto(Producto("Lechuga", "2024-10-30", perecible=True))
verduras.agregar_producto(Producto("Rabanito", "2024-11-10", perecible=True))
verduras.agregar_producto(Producto("Brocoli", "2024-11-05", perecible=True))
conservas.agregar_producto(Producto("Filete de atún en aceite vegetal", perecible=False))
conservas.agregar_producto(Producto("Conserva de piña en almíbar", perecible=False))
conservas.agregar_producto(Producto("Conserva de durazno", perecible=False))
granos.agregar_producto(Producto("Avena", perecible=False))
granos.agregar_producto(Producto("Arroz", perecible=False))
granos.agregar_producto(Producto("Quinoa", perecible=False))
granos.agregar_producto(Producto("Pasta", perecible=False))

In [39]:
class NodoProducto:
    def __init__(self, producto):
        self.producto = producto  # Almacena el producto
        self.siguiente = None  # Puntero al siguiente nodo

In [60]:
class PilaProductosPerecibles:
    def __init__(self):
        self.cima = None

    def esta_vacia(self):
        return self.cima is None

    def push(self, producto):
        if not producto.perecible:
            raise ValueError("Solo se pueden agregar productos perecibles a la pila.")
        
        nuevo_nodo = NodoProducto(producto)
        nuevo_nodo.siguiente = self.cima
        self.cima = nuevo_nodo

    def pop(self):
        if self.esta_vacia():
            raise IndexError("La pila está vacía")
        producto = self.cima.producto
        self.cima = self.cima.siguiente
        return producto

    def peek(self):
        if self.esta_vacia():
            raise IndexError("La pila está vacía")
        return self.cima.producto

    def mostrar_pila(self):
        nodos = []
        actual = self.cima
        while actual:
            nodos.append(actual.producto.nombre)
            actual = actual.siguiente
        print(" -> ".join(nodos))

    def llenar_desde_arbol(self, arbol):
        """Llena la pila con productos perecibles desde el árbol."""
        productos_perecibles = arbol.buscar_productos_perecibles()
        for producto in productos_perecibles:
            self.push(producto)
    
    def filtrar_por_fecha(self, fecha_ingresada):
        fecha_busqueda = datetime.strptime(fecha_ingresada, "%Y-%m-%d")
        actual = self.cima
        print(f"\nProductos que vencen después de {fecha_ingresada}:")

        while actual:
            if actual.producto.fecha_vencimiento > fecha_busqueda:
                print(f"Producto: {actual.producto.nombre}, Fecha de vencimiento: {actual.producto.fecha_vencimiento.strftime('%Y-%m-%d')}")
            actual = actual.siguiente
    
    def mostrar_menor_fecha(self, fecha_ingresada):
        fecha_busqueda = datetime.strptime(fecha_ingresada, "%Y-%m-%d")
        actual = self.cima
        print(f"\nProductos que vencen antes de {fecha_ingresada}:")

        while actual:
            if actual.producto.fecha_vencimiento < fecha_busqueda:
                print(f"Producto: {actual.producto.nombre}, Fecha de vencimiento: {actual.producto.fecha_vencimiento.strftime('%Y-%m-%d')}")
            actual = actual.siguiente

In [61]:
pila_perecibles = PilaProductosPerecibles()
pila_perecibles.llenar_desde_arbol(arbol_productos)

In [65]:
print("Productos perecibles en la pila:")
pila_perecibles.mostrar_pila()

Productos perecibles en la pila:
Rabanito -> Brocoli -> Lechuga -> Platano -> Melocoton -> Manzana


In [66]:
fecha_ingresada = input("Ingrese la fecha de vencimiento que desea ver: ")
pila_perecibles.filtrar_por_fecha(fecha_ingresada)


Productos que vencen después de 2024-11-15:
Producto: Platano, Fecha de vencimiento: 2024-12-01


In [67]:
fecha_ingresada = input("Ingrese la fecha de vencimiento que desea ver: ")
pila_perecibles.mostrar_menor_fecha(fecha_ingresada)


Productos que vencen antes de 2024-11-15:
Producto: Rabanito, Fecha de vencimiento: 2024-11-10
Producto: Brocoli, Fecha de vencimiento: 2024-11-05
Producto: Lechuga, Fecha de vencimiento: 2024-10-30
Producto: Manzana, Fecha de vencimiento: 2024-11-10


# **Prueba 2:**